# Code for Protects

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

# Set the root
import sys
from pathlib import Path
from sysconfig import get_path

import pandas as pd
from from_root import from_root
from ib_async import util

MARKET = "SNP"

ROOT = from_root()

pd.options.display.max_columns = None
pd.options.display.float_format = '{:20,.2f}'.format


# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
util.startLoop()

In [ ]:
import asyncio
from ibfuncs import df_chains, df_iv, get_ib, get_open_orders, quick_pf
from snp import get_snp_unds

unds = get_snp_unds()

with get_ib('SNP') as ib:

    unds_iv=ib.run(df_iv(ib=ib, stocks=unds, msg='first run ivs'))

    no_price=unds_iv[unds_iv[['price', 'hv', 'iv']].isnull().any(axis=1)].symbol.to_list()
    second_unds_iv = ib.run(df_iv(ib=ib, stocks=no_price, sleep_time=10, msg='second run ivs'))

    pf_raw = quick_pf(ib)
    oo = get_open_orders(ib)

    # Set symbol as index for both dataframes
    cols = ['symbol', 'price', 'iv', 'hv']

    unds_iv = unds_iv[cols].set_index('symbol')
    second_unds_iv = second_unds_iv[cols].set_index('symbol')

    # Update unds_iv with non-null values from second_unds_iv 
    unds_iv.update(second_unds_iv)

    # unds_iv = unds_iv.set_index('symbol')[['hv', 'iv', 'price']]
    unds_iv.columns = ['und_' + col for col in unds_iv.columns]
    unds_iv = unds_iv.reset_index()

    # ... add und_price
    pf = pf_raw.merge(unds_iv, on='symbol', how='left')

    # Update und_price with mktPrice where und_price is NaN and mktPrice has a value
    pf.loc[pd.isna(pf['und_price']) & pd.notna(pf['mktPrice']), 'und_price'] = pf['mktPrice']

    # Merge the DataFrames on 'symbol'
    unds_iv = unds_iv.merge(pf[['symbol', 'und_price']], on='symbol', how='left')

    # Fill NaN values in 'und_price_x' with values from 'und_price_y'
    unds_iv['und_price'] = unds_iv['und_price_x'].fillna(unds_iv['und_price_y'])

    # Drop the unnecessary 'und_price_x' and 'und_price_y' columns
    unds_iv = unds_iv.drop(columns=['und_price_x', 'und_price_y'])

    df = pf


In [20]:
# Group by symbol and get strategy based on secType priority
def get_strategy(group):
    stk_rows = group[group['secType'] == 'STK']
    if not stk_rows.empty:
        return stk_rows['strategy'].iloc[0]
    opt_rows = group[group['secType'] == 'OPT'] 
    if not opt_rows.empty:
        return opt_rows['strategy'].iloc[0]
    return 'NA'

summary = df.groupby('symbol').agg({
    'unPnL': 'sum',
    'secType': 'first',
    'position': 'first'
}).sort_values('unPnL')


In [ ]:
df.apply(get_strategy)

In [ ]:
# Add strategy based on secType priority
summary['strategy'] = df.groupby('symbol').apply(get_strategy, include_groups=False)

summary['position_type'] = summary.apply(lambda x: 
    'long stock' if x['secType'] == 'STK' and x['position'] > 0 
    else 'short stock' if x['secType'] == 'STK' and x['position'] < 0
    else 'NA', axis=1)

In [ ]:
get_strategy(df)

In [ ]:
# Add strategy based on secType priority
summary['strategy'] = df.groupby('symbol').apply(get_strategy, include_groups=False)

summary['position_type'] = summary.apply(lambda x: 
    'long stock' if x['secType'] == 'STK' and x['position'] > 0 
    else 'short stock' if x['secType'] == 'STK' and x['position'] < 0
    else 'NA', axis=1)

# Display results
print("\nP&L, Position Type and Strategy by Symbol:")
print(summary[['unPnL', 'position_type', 'strategy']].to_string(float_format=lambda x: f"${x:,.2f}"))

m = (df.strategy == 'unprotected') | (df.strategy == 'exposed')
df = df[m].reset_index(drop=True)

In [ ]:

    chains = asyncio.run(df_chains(ib, unds, msg='raw chains'))
